In [30]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

In [31]:
# dimensions of our images.
img_width, img_height = 300, 300

In [32]:
top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'Pavement/train'
validation_data_dir = 'Pavement/validation'
nb_train_samples = 24
nb_validation_samples = 4
epochs = 50
batch_size = 2

In [33]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)

In [34]:
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
    train_labels = np.array(
        [0] * int(nb_train_samples / 2) + [1] * int(nb_train_samples / 2))

    validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
    validation_labels = np.array(
        [0] * int(nb_validation_samples / 2) + [1] * int(nb_validation_samples / 2))
    
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)
    
    #test
    scores = model.evaluate(test_data, test_labels, verbose=0)

In [14]:
save_bottlebeck_features()

Found 24 images belonging to 2 classes.
Found 4 images belonging to 2 classes.


In [19]:
train_top_model()

Train on 24 samples, validate on 4 samples
Epoch 1/50
24/24 [==============================] - 1s 56ms/step - loss: 7.2765 - accuracy: 0.5417 - val_loss: 3.0043 - val_accuracy: 0.5000
Epoch 2/50
24/24 [==============================] - 1s 36ms/step - loss: 3.8883 - accuracy: 0.5833 - val_loss: 0.3620 - val_accuracy: 0.7500
Epoch 3/50
24/24 [==============================] - 1s 37ms/step - loss: 1.2969 - accuracy: 0.7083 - val_loss: 0.2616 - val_accuracy: 0.7500
Epoch 4/50
24/24 [==============================] - 1s 38ms/step - loss: 1.2329 - accuracy: 0.7917 - val_loss: 1.9924 - val_accuracy: 0.5000
Epoch 5/50
24/24 [==============================] - 1s 37ms/step - loss: 0.9868 - accuracy: 0.8750 - val_loss: 0.0226 - val_accuracy: 1.0000
Epoch 6/50
24/24 [==============================] - 1s 41ms/step - loss: 0.4200 - accuracy: 0.9167 - val_loss: 0.3922 - val_accuracy: 0.7500
Epoch 7/50
24/24 [==============================] - 1s 36ms/step - loss: 0.5109 - accuracy: 0.8333 - val_loss: 

In [39]:
from keras.models import load_model

model = load_model('bottleneck_fc_model.h5')

ValueError: Cannot create group in read-only mode.